In [ ]:
FDAI_API_KEY = 'fe0aabe5-b190-4b38-9b06-d937aa9a1aee'
BACKTEST_DATE = '2025-06-06'

In [2]:
import requests
import pandas as pd
import numpy as np

## Data Retrieval

In [3]:
def load_daily_price_data(ticker, start_date, end_date, interval='day', interval_multiplier=1):
    """
    Load daily price data from financialdatasets.ai for given tickers and date range.

    Args:
        tickers (list of str): List of ticker symbols.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
        dict: Dictionary mapping tickers to their price data (as lists of dicts).
    """

    headers = {
        "X-API-KEY": FDAI_API_KEY
    }

    # create the URL
    url = (
        f'https://api.financialdatasets.ai/prices/'
        f'?ticker={ticker}'
        f'&interval={interval}'
        f'&interval_multiplier={interval_multiplier}'
        f'&start_date={start_date}'
        f'&end_date={end_date}'
    )

    # make API request
    response = requests.get(url, headers=headers)

    # parse prices from the response
    response.raise_for_status()
    return pd.DataFrame(response.json()['prices'])

In [4]:
def load_fundamental_metrics(ticker, period, limit=5):
    
    # add your API key to the headers
    headers = {
        "X-API-KEY": FDAI_API_KEY
    }

      # number of periods to return

    # create the URL
    url = (
        f'https://api.financialdatasets.ai/financial-metrics'
        f'?ticker={ticker}'
        f'&period={period}'
        f'&limit={limit}'
    )

    # make API request
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    # parse financial_metrics from the response
    financial_metrics = response.json().get('financial_metrics')
    return pd.DataFrame(financial_metrics)

In [5]:
tickers = ["AAPL", "MSFT", "NVDA", "TSLA"]

In [6]:
start_date = "2025-06-01"
end_date = "2025-09-12"

## Valuation Agent

In [7]:
df_list =[]
for i in tickers:
    data = load_daily_price_data(i,start_date,end_date)
    df_list.append(data)
stock_data = pd.concat(df_list, ignore_index=True)
stock_data.to_csv('C:/Working/Agent1/stock_data.csv', index=False)

In [8]:
stock_data['date'] = pd.to_datetime(stock_data['time']).dt.date
stock_data['close_pct_change'] = stock_data.groupby('ticker')['close'].pct_change()

In [9]:
stock_data['annualised_7_day_momentum'] = (1 + stock_data['close_pct_change']) ** (252/7) - 1

In [10]:
stock_data['vol_7_daily_unannualized'] = stock_data['close_pct_change'].rolling(7).std(ddof=1)

# annualized 7-day volatility
stock_data['vol_7_annualized'] = stock_data['vol_7_daily_unannualized'] * np.sqrt(252)

In [11]:
stock_data['sharpe_ratio_7_day'] = stock_data['annualised_7_day_momentum'] / stock_data['vol_7_annualized']

In [12]:
stock_data['signal'] = stock_data['sharpe_ratio_7_day'].apply(lambda x: 'buy' if x > 1 else ('sell' if x < -1 else 'hold'))

## Sentiment Agent

In [13]:
## News/Sentiment Data
## Finbert

news_data_aapl = pd.read_json('C:/Working/Agent1/Data/AAPL.json')
news_data_msft = pd.read_json('C:/Working/Agent1/Data/MSFT.json')
news_data_nvda = pd.read_json('C:/Working/Agent1/Data/NVDA.json')
news_data_tsla = pd.read_json('C:/Working/Agent1/Data/TSLA.json')
news_data_all = pd.concat([news_data_aapl, news_data_msft, news_data_nvda, news_data_tsla], ignore_index=True)

In [14]:
import torch
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

c:\Users\nedgu\miniconda3\envs\antipodes_agent1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [15]:
news_data_all['sentiment'] = news_data_all['summary'].apply(lambda x: pipe(x)[0]['label'])

In [16]:
news_data_all['signal'] = news_data_all['sentiment'].map({'positive': 'buy', 'negative': 'sell', 'neutral': 'hold'})
news_data_all

,ticker,title,date,summary,url,sentiment,signal
0,AAPL,Apple weighs using Anthropic or OpenAI to powe...,2025-07-01,Apple is exploring using AI models from Anthro...,https://www.reuters.com/business/apple-weighs-...,negative,sell
1,AAPL,"Apple internally discussed buying Mistral, Per...",2025-08-27,"Apple (AAPL.O), opens new tab has held talks i...",https://www.reuters.com/business/apple-interna...,positive,buy
2,AAPL,Apple's tariff-fueled iPhone sales surge raise...,2025-08-02,Apple's best revenue growth in three years fai...,https://www.reuters.com/business/apples-tariff...,negative,sell
3,AAPL,Musk says xAI to take legal action against App...,2025-08-13,Billionaire Elon Musk said on Monday his arti...,https://www.reuters.com/sustainability/boards-...,negative,sell
4,AAPL,"Apple Posts Big Sales Beat on iPhone Demand, C...",2025-08-01,Apple’s quarterly revenue handily topped analy...,https://www.bloomberg.com/news/newsletters/202...,positive,buy
5,AAPL,"JPMorgan Bumps Apple (AAPL) PT to $250, Keeps ...",2025-07-20,JPMorgan raised Apple’s price target to $250 a...,https://finance.yahoo.com/news/jpmorgan-bumps-...,positive,buy
6,AAPL,Apple is developing specialized chips for smar...,2025-05-09,"Apple (AAPL.O), opens new tab is developing sp...",https://www.reuters.com/world/china/apple-is-d...,positive,buy
7,MSFT,Microsoft and Meta fuel $500-billion gain in A...,2025-07-31,Wall Street's AI heavyweights added a combined...,https://www.reuters.com/business/retail-consum...,positive,buy
8,MSFT,Microsoft sued by authors over use of books in...,2025-06-26,"A group of authors sued Microsoft, alleging it...",https://www.reuters.com/sustainability/boards-...,negative,sell
9,MSFT,Microsoft knew of SharePoint security flaw but...,2025-07-23,"LONDON, July 22 (Reuters) - A security patch M...",https://www.reuters.com/sustainability/boards-...,negative,sell


## Fundamental Agent

In [17]:
df_list = []
for i in tickers:
    data = load_fundamental_metrics(i,"quarterly",5)
    df_list.append(data)
fundamental_data = pd.concat(df_list, ignore_index=True)

fundamental_data['report_period'] = pd.to_datetime(fundamental_data['report_period'])
fundamental_fields = ['ticker', 'report_period', 'fiscal_period', 'period', 'currency','operating_income_growth','operating_margin','operating_cash_flow_ratio', 'receivables_turnover']
fundamental_data_subset = fundamental_data[fundamental_fields]
fundamental_data_subset.sort_values(['ticker', 'report_period'],ascending=[True, True],inplace=True)

C:\Users\nedgu\AppData\Local\Temp\ipykernel_18176\80090463.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fundamental_data = pd.concat(df_list, ignore_index=True)
C:\Users\nedgu\AppData\Local\Temp\ipykernel_18176\80090463.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamental_data_subset.sort_values(['ticker', 'report_period'],ascending=[True, True],inplace=True)


In [18]:
# Calculate % change for each ticker and period for selected columns
cols = ['operating_margin', 'operating_cash_flow_ratio', 'receivables_turnover']
pct_change_df = (
    fundamental_data_subset
    .groupby('ticker')[cols]
    .pct_change()
    .rename(lambda x: f'{x}_pct_change', axis=1)
    .reset_index(drop=True)
    
)
result = pd.concat([fundamental_data_subset.reset_index(drop=True), pct_change_df], axis=1)
result.to_csv('C:/Working/Agent1/fundamental_data.csv', index=False)

In [ ]:
cutoff_date = pd.to_datetime(BACKTEST_DATE)
filtered = fundamental_data_subset[fundamental_data_subset['report_period'] < cutoff_date]
latest_reports_before_cutoff = filtered.groupby('ticker')['report_period'].max().reset_index()
merged = pd.merge( latest_reports_before_cutoff,  result, on=['ticker', 'report_period'], how='left')


In [20]:
import math
def sigmoid(x, k=10):
    """Sigmoid scaling for percentage values."""
    return 1 / (1 + math.exp(-x / k))

def fundamental_signal_df(df):
    """
    Compute composite score and signal for each row in a DataFrame using fundamental metrics.
    Expects columns:
      - operating_income_growth
      - operating_margin_pct_change
      - operating_cash_flow_ratio_pct_change
      - receivables_turnover_pct_change
    Returns:
      DataFrame with 'fundamental_signal' and 'fundamental_score' columns.
    """
    weights = {
        'income': 0.30,
        'margin': 0.20,
        'ocf': 0.40,
        'receivables': 0.10
    }

    def row_signal(row):
        s_income = sigmoid(row['operating_income_growth'], k =10)
        s_margin = sigmoid(row['operating_margin_pct_change'], k=3)
        s_ocf = sigmoid(row['operating_cash_flow_ratio_pct_change'], k=5)
        s_receivables = sigmoid(row['receivables_turnover_pct_change'], k =3)
        score = (weights['income'] * s_income +
                 weights['margin'] * s_margin +
                 weights['ocf'] * s_ocf +
                 weights['receivables'] * s_receivables)
        if score > 0.67:
            signal = "Buy"
        elif score < 0.33:
            signal = "Sell"
        else:
            signal = "Hold"
        return pd.Series({'fundamental_signal': signal, 'fundamental_score': score})

    return df.apply(row_signal, axis=1)

In [21]:
signal_result = fundamental_signal_df(merged)
fundamental_signal_df = pd.concat([merged, signal_result], axis=1)
fundamental_signal_df

,ticker,report_period,fiscal_period,period,currency,operating_income_growth,operating_margin,operating_cash_flow_ratio,receivables_turnover,operating_margin_pct_change,operating_cash_flow_ratio_pct_change,receivables_turnover_pct_change,fundamental_signal,fundamental_score
0,AAPL,2025-03-29,2025-Q2,quarterly,USD,-0.311713,0.307365,0.165676,1.748039,-0.102822,-0.201006,-0.117198,Hold,0.490953
1,MSFT,2025-03-31,2025-Q3,quarterly,USD,0.067158,0.456298,0.324361,1.402891,0.060548,0.584366,-0.069843,Hold,0.512605
2,NVDA,2025-04-27,2026-Q1,quarterly,USD,-0.130746,0.498684,1.032854,1.949775,-0.224079,0.120928,0.010258,Hold,0.497790
3,TSLA,2025-03-31,2025-Q1,quarterly,USD,-0.765016,0.034600,0.072463,4.715854,-0.687575,-0.566169,-0.290888,Hold,0.469122


## Coordinator

In [ ]:
## Simple equal weighted average of the recommendations


In [45]:
news_data_all['date'] = pd.to_datetime(news_data_all['date'])
filled_list = []

for ticker in tickers:
    df_ticker = news_data_all[news_data_all['ticker'] == ticker].copy()
    full_dates = pd.date_range(start='2025-05-01', end='2025-08-31', freq='D')
    df_ticker = df_ticker.set_index('date').reindex(full_dates)
    df_ticker['ticker'] = ticker
    df_ticker['signal'] = df_ticker['signal'].ffill()
    df_ticker = df_ticker.reset_index().rename(columns={'index': 'date'})
    filled_list.append(df_ticker)

news_data_all_filled = pd.concat(filled_list, ignore_index=True)
news_data_all_filled['date'] = pd.to_datetime(news_data_all_filled['date']).dt.date

In [48]:

fundamental_signal_df['report_period'] = pd.to_datetime(fundamental_signal_df['report_period'])
filled_list = []

for ticker in tickers:
    df_ticker = fundamental_signal_df[fundamental_signal_df['ticker'] == ticker].copy()
    full_dates = pd.date_range(start='2025-03-27', end='2025-08-31', freq='D')
    df_ticker = df_ticker.set_index('report_period').reindex(full_dates)
    df_ticker['ticker'] = ticker
    df_ticker['fundamental_signal'] = df_ticker['fundamental_signal'].ffill()
    df_ticker['fundamental_score'] = df_ticker['fundamental_score'].ffill()
    df_ticker = df_ticker.reset_index().rename(columns={'index': 'report_period'})
    filled_list.append(df_ticker)

fundamental_signal_df_filled = pd.concat(filled_list, ignore_index=True)
fundamental_signal_df_filled['date'] = pd.to_datetime(fundamental_signal_df_filled['report_period']).dt.date

In [50]:
fundamental_signal_df_filled

,report_period,ticker,fiscal_period,period,currency,operating_income_growth,operating_margin,operating_cash_flow_ratio,receivables_turnover,operating_margin_pct_change,operating_cash_flow_ratio_pct_change,receivables_turnover_pct_change,fundamental_signal,fundamental_score,date
0,2025-03-27,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-27
1,2025-03-28,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-28
2,2025-03-29,AAPL,2025-Q2,quarterly,USD,-0.311713,0.307365,0.165676,1.748039,-0.102822,-0.201006,-0.117198,Hold,0.490953,2025-03-29
3,2025-03-30,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.490953,2025-03-30
4,2025-03-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.490953,2025-03-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,2025-08-27,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.469122,2025-08-27
628,2025-08-28,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.469122,2025-08-28
629,2025-08-29,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.469122,2025-08-29
630,2025-08-30,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hold,0.469122,2025-08-30


In [51]:
combined_signal_df = pd.merge(stock_data[['ticker','close','date','signal']], news_data_all_filled[['date', 'ticker', 'signal']], on=['date', 'ticker'], how='inner', suffixes=('_price', '_news'))
combined_signal_df = pd.merge(combined_signal_df, fundamental_signal_df_filled[['date', 'ticker', 'fundamental_signal']], on=['date', 'ticker'], how='inner')

In [60]:
def combine_signals(*signals):
    """
    Combine multiple signals ('buy', 'hold', 'sell') into a final signal.
    Returns 'buy' if sum > 0, 'hold' if sum == 0, 'sell' if sum < 0.
    """
    mapping = {'buy': 1, 'hold': 0, 'sell': -1}
    total = 0
    for sig in signals:
        if isinstance(sig, str):
            total += mapping.get(sig.lower(), 0)
    if total > 0:
        return 'buy'
    elif total < 0:
        return 'sell'
    else:
        return 'hold'

In [61]:
combined_signal_df['final_signal'] = combined_signal_df.apply(
    lambda row: combine_signals(row['signal_price'], row['signal_news'], row['fundamental_signal']),
    axis=1
)

## Backtest

In [ ]:
backtest_as_of_date = "2025-06-06"

ticker
AAPL   2025-05-09
MSFT   2025-05-13
NVDA   2025-05-27
TSLA   2025-06-06
Name: date, dtype: datetime64[ns]